In [19]:
# Make the cells wider in the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from importlib import reload
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import waveglow_model as model
import pandas as pd

In [12]:
# Import the data
trainset = pd.read_pickle("wind_power_data/wind_power_train.pickle").values
testset = pd.read_pickle("wind_power_data/wind_power_test.pickle").values

In [28]:
# Try instantiating a network
reload(model)
net = model.WaveGlow(
    n_context_channels=96, 
    n_flows=6, 
    n_group=24, 
    n_early_every=3,
    n_early_size=8,
    n_layers=2,
    dilation_list=[1,2],
    n_channels=96,
    kernel_size=3);

Channels:  24
Channels:  24
Channels:  24
Channels:  16
Channels:  16
Channels:  16


In [31]:
trainset.shape

(70080,)

In [ ]:
net()